In [4]:
import numpy as np
from pandas import DataFrame
import pandas as pd
from matplotlib import pyplot
from sklearn.preprocessing import KBinsDiscretizer
import seaborn as sns
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import _tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import accuracy_score
import warnings; warnings.simplefilter('ignore')

In [2]:
df=pd.read_csv("Dry_Bean.csv")
df.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [3]:
df.columns

Index(['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength',
       'AspectRation', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent',
       'Solidity', 'roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2',
       'ShapeFactor3', 'ShapeFactor4', 'Class'],
      dtype='object')

# GA


In [7]:
X=df.iloc[:, :-1]
Y=df['Class']

In [8]:
def initialize_population():
    population = np.random.randint(2, size=(10,16))
    features = ['Area','Perimeter','MajorAxisLength','MinorAxisLength','AspectRation','Eccentricity','ConvexArea','EquivDiameter','Extent','Solidity','roundness','Compactness','ShapeFactor1','ShapeFactor2','ShapeFactor3','ShapeFactor4']
    return population,features

In [9]:
#FITNESS FUNCTION
def cost(features,chromosome):
    feature_selected = []
    for i in range(len(features)):
        if chromosome[i] == 1:
            feature_selected.append(features[i])
    x = df[feature_selected]
    x_abs=x.abs()
    xMax = x_abs.max()
    x = x/xMax
    y = df[['Class']]
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3,random_state =2)
    clf = DecisionTreeClassifier(criterion="entropy",random_state=2,max_depth=20)
    clf.fit(x_train, y_train)
    t=clf.predict(x_test)
    return accuracy_score(y_test, t)

In [10]:
def random_combine(parents,n_offspring):
    offspring = []
    n_parents = parents.shape[0]
    for i in range(n_offspring):
        random_dad = parents[np.random.randint(low =0,high = n_parents - 1)]
        random_mom = parents[np.random.randint(low =0,high = n_parents - 1)]
        dad_mask = np.random.randint(0,2,np.array(random_dad).shape)
        mom_mask = np.logical_not(dad_mask)
        child = np.add(np.multiply(random_dad,dad_mask),np.multiply(random_mom,mom_mask))
        offspring.append(child)
    offspring = np.array(offspring)
    return offspring

In [11]:
def mutate_parent(parent,n_mutations):
    size1 = parent.shape[0]
    for i in range(n_mutations):
        rand1 = np.random.randint(0,size1)
        if parent[rand1] == 0:
            parent[rand1] = 1
        else:
            parent[rand1] = 0
    return parent
def mutate_gen(parent_gen,n_mutations):
    mutated_parent_gen = []
    for parent in parent_gen:
        mutated_parent_gen.append(mutate_parent(parent,n_mutations))
    mutated_parent_gen = np.array(mutated_parent_gen)
    #print("Mutated population:",mutated_parent_gen)
    return mutated_parent_gen

In [12]:
def select_best(features,parent_gen):
    costs = []
    selected_parent = []
    for i in range(len(parent_gen)):
        costs.append(cost(features,parent_gen[i]))
        #print("Cost",[i],"=",costs[i])
        if costs[i] > 0.4:
            selected_parent.append(parent_gen[i])
            #print("Selected",[i],"=",selected_parent[i])
    #print(costs) 
    costs = np.array(costs)
    selected_parent = np.array(selected_parent)
    #print(selected_parent)
    #print(costs)
    #print(selected_parent.shape)
    max_index = np.where(costs == np.amax(costs))
    feasible_features = (parent_gen[max_index])[0]
    #print("Feasible feature:",feasible_features)
    return selected_parent,costs,feasible_features

In [15]:
def gen_algo():
    parent_gen,features = initialize_population()
    print(parent_gen.shape)
    generations = 15
    overall_costs = []
    overall_features = []
    for i in range(generations):
        print("Generation number :",i+1)
        parent_gen,costs,feasible_features = select_best(features,parent_gen)
        print("Accuracy for this generation:",np.amax(costs))
        print("Feasible features for this generation:",feasible_features)
        overall_costs.append(np.amax(costs))
        overall_features.append(feasible_features)
        if parent_gen.shape[0] <= 1:
            parent_gen,features = initialize_population()
        else :
            parent_gen = random_combine(parent_gen,20)
        parent_gen = mutate_gen(parent_gen,2)
        print("=============================================================================================================")
    overall_costs = np.array(overall_costs)
    overall_features = np.array(overall_features)
    #best_child,costs,feasible_features = select_best(features,parent_gen)
    max_index = np.where(overall_costs == np.amax(overall_costs))
    print("Final cost:",np.amax(overall_costs))
    print("Final features:",(overall_features[max_index])[0])
    return (overall_features[max_index])[0]

In [16]:
final_feature = gen_algo()

(10, 16)
Generation number : 1
Accuracy for this generation: 0.8959353574926543
Feasible features for this generation: [1 0 0 1 0 0 1 1 0 1 1 1 1 1 0 0]
Generation number : 2
Accuracy for this generation: 0.8976493633692458
Feasible features for this generation: [1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 1]
Generation number : 3
Accuracy for this generation: 0.8981390793339863
Feasible features for this generation: [1 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1]
Generation number : 4
Accuracy for this generation: 0.8974045053868757
Feasible features for this generation: [1 1 0 0 1 1 0 1 1 1 1 1 0 0 0 1]
Generation number : 5
Accuracy for this generation: 0.9005876591576886
Feasible features for this generation: [1 0 1 0 1 1 1 1 0 1 1 1 0 0 1 1]
Generation number : 6
Accuracy for this generation: 0.8983839373163565
Feasible features for this generation: [1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 1]
Generation number : 7
Accuracy for this generation: 0.8961802154750245
Feasible features for this generation: [1 0 1 0 1 0 1 1 

In [17]:
to_drop =['Perimeter','MajorAxisLength','MinorAxisLength','AspectRation','Eccentricity','Extent','ShapeFactor2']
df.drop(columns= to_drop, axis=1, inplace=True) 
df.head()

,Area,ConvexArea,EquivDiameter,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor3,ShapeFactor4,Class
0,28395,28715,190.141097,0.988856,0.958027,0.913358,0.007332,0.834222,0.998724,SEKER
1,28734,29172,191.272751,0.984986,0.887034,0.953861,0.006979,0.909851,0.998430,SEKER
2,29380,29690,193.410904,0.989559,0.947849,0.908774,0.007244,0.825871,0.999066,SEKER
3,30008,30724,195.467062,0.976696,0.903936,0.928329,0.007017,0.861794,0.994199,SEKER
4,30140,30417,195.896503,0.990893,0.984877,0.970516,0.006697,0.941900,0.999166,SEKER


In [18]:
X=df.iloc[:, :-1]
Y=df['Class']
kbins = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
data_trans = kbins.fit_transform(X)
df1 = DataFrame(data_trans)
df1.columns=('Area','ConvexArea','EquivDiameter','Solidity','roundness','Compactness','ShapeFactor1','ShapeFactor3','ShapeFactor4')
df1.head()

,Area,ConvexArea,EquivDiameter,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor3,ShapeFactor4
0,0.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,2.0
1,0.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,2.0
2,0.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,2.0
3,0.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,2.0
4,0.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,2.0


In [19]:
X=df1
Y=df['Class']

In [20]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,test_size = 0.3,random_state =2)
DecisionTree = DecisionTreeClassifier(criterion="entropy",random_state=2,max_depth=20)

DecisionTree.fit(Xtrain,Ytrain)

#predicted_values = DecisionTree.predict(Xtest)
#x = metrics.accuracy_score(Ytest, predicted_values)
#acc.append(x)
#model.append('Decision Tree')
#print("DecisionTrees's Accuracy is: ", x*100)

DecisionTrees's Accuracy is:  73.45739471106758


In [21]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        if(path[-1][1]>100):
            rule = "if "
            for p in path[:-1]:
                if rule != "if ":
                    rule += " and "
                rule += str(p)
            rule += " then "
            if class_names is None:
                rule += "response: "+str(np.round(path[-1][0][0][0],3))
            else:
                classes = path[-1][0][0]
                l = np.argmax(classes)
                rule += f"class: {class_names[l]}(proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
            rule += f" | based on {path[-1][1]:,} samples"
            rules += [rule]

    return rules

In [23]:
feature_cols = ['Area','ConvexArea','EquivDiameter','Solidity','roundness','Compactness','ShapeFactor1','ShapeFactor3','ShapeFactor4']
rules = get_rules(DecisionTree,feature_cols,Y.unique())
c=0
for r in rules:
    c=c+1
    print(r)
print(c)

if (ShapeFactor3 > 0.5) and (Compactness <= 1.5) and (EquivDiameter <= 0.5) and (ShapeFactor1 <= 1.5) and (roundness > 1.5) and (ShapeFactor1 > 0.5) and (ShapeFactor4 > 1.5) then class: DERMASON(proba: 44.62%) | based on 3,395 samples
if (ShapeFactor3 <= 0.5) and (EquivDiameter <= 0.5) and (roundness <= 1.5) and (Compactness <= 0.5) and (ShapeFactor1 <= 1.5) and (Solidity > 1.5) and (ShapeFactor4 > 1.5) and (roundness > 0.5) then class: HOROZ(proba: 93.66%) | based on 1,056 samples
if (ShapeFactor3 > 0.5) and (Compactness > 1.5) and (ShapeFactor1 > 0.5) and (ShapeFactor1 <= 1.5) and (ShapeFactor3 > 1.5) and (roundness > 1.5) and (Solidity > 1.5) then class: SIRA(proba: 98.49%) | based on 930 samples
if (ShapeFactor3 > 0.5) and (Compactness <= 1.5) and (EquivDiameter <= 0.5) and (ShapeFactor1 > 1.5) and (roundness > 1.5) and (ShapeFactor4 > 1.5) then class: CALI(proba: 99.88%) | based on 860 samples
if (ShapeFactor3 <= 0.5) and (EquivDiameter <= 0.5) and (roundness > 1.5) and (ShapeFact